In [ ]:
import numpy as np
import h5py
import numpy as np
from scipy.integrate import solve_ivp
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import sys
import torch
import matplotlib.pyplot as plt

In [ ]:
psi = np.load('/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/data/actual/long_run_310k.npy')

In [ ]:
# Pre-processing

lead = 1

trainN = 200000
valN = 50000
index = 63

psi = psi[:,1,:]

print(psi.shape)
mean_psi = np.mean(psi, axis=0, keepdims=True)
std_psi = np.std(psi, axis=0, keepdims=True)
psi = (psi - mean_psi) / std_psi

psi_train_input = torch.tensor(psi[0:trainN,:])
psi_train_label =  torch.tensor(psi[lead:trainN+lead,:])

psi_val_input = torch.tensor(psi[trainN:trainN+valN,:])
psi_val_label =  torch.tensor(psi[trainN+lead:trainN+valN+lead,:])

print(psi_train_input.shape)
print(psi_train_label.shape)
print(psi_val_input.shape)
print(psi_val_label.shape)

In [ ]:
print(np.shape(psi))

In [ ]:
plt.plot(psi_train_input[0:200000,63])

In [ ]:
plt.plot(psi_val_input[0:200000,63])

In [ ]:
# Define the encoder (MLP)
class Encoder(nn.Module):
    def __init__(self, input_dim, latent_dim):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(75, 512)  # Input layer (2 + 2) -> Hidden layer (128)
        self.fc2 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc3 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc4 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc5 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc6 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc_mu = nn.Linear(512, latent_dim)  # Hidden layer (128) -> Latent space (2)
        self.fc_logvar = nn.Linear(512, latent_dim)  # Hidden layer (128) -> Log variance (2)

    def forward(self, x):
        x = torch.relu(self.fc1(x))  # Activation function for hidden layer
        x = torch.relu(self.fc2(x)) + x
        x = torch.relu(self.fc3(x)) + x
        x = torch.relu(self.fc4(x)) + x
        mu = self.fc_mu(x)
        logvar = self.fc_logvar(x)
        return mu, logvar

# Define the decoder (MLP)
class Decoder(nn.Module):
    def __init__(self, latent_dim, output_dim, condition_dim):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(latent_dim + condition_dim, 512)  # Input layer (2 + 2) -> Hidden layer (128)
        self.fc2 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc3 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc4 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc5 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc6 = nn.Linear(512, 512)  # Hidden layer (128) -> Hidden layer (128)
        self.fc_output = nn.Linear(512, output_dim)  # Hidden layer (128) -> Output layer (2)

    def forward(self, z, condition):
        z = torch.cat((z, condition), dim=1)  # Concatenate latent vector and condition
        z = torch.relu(self.fc1(z))  # Activation function for hidden layer
        z = torch.relu(self.fc2(z)) + z
        z = torch.relu(self.fc3(z)) + z
        z = torch.relu(self.fc4(z)) + z
        output = self.fc_output(z)
        return output

# Define the VAE model
class ConditionalVAE(nn.Module):
    def __init__(self, input_dim, latent_dim, output_dim, condition_dim):
        super(ConditionalVAE, self).__init__()
        self.encoder = Encoder(input_dim, latent_dim)
        self.decoder = Decoder(latent_dim, output_dim, condition_dim)

    def encode(self, x):
        mu, logvar = self.encoder(x)
        return mu, logvar

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        z = mu + eps * std
        return z

    def decode(self, z, condition):
        return self.decoder(z, condition)

    def forward(self, x, condition):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        output = self.decode(z, condition)
        return output, mu, logvar

input_dim = 1
output_dim = 75
latent_dim = 1024
condition_dim = 75
batch_size = 1024
num_epochs = 100

In [ ]:
# Training

import datetime
import os

# Initialize the model, optimizer, and loss function
model = ConditionalVAE(input_dim, latent_dim, output_dim, condition_dim)
model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

folder = f"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_at_{datetime.datetime.now()}"
model_weights_path = f'{folder}/model_weights_pytorch.pth'
os.makedirs(folder)

# from torchsummary import summary

# summary(model, input_size = [(128, 1, 75), (128, 75)])
# Train the model
for epoch in range(num_epochs):
    model.train()
    for batch in range (0, trainN, batch_size):

        input_batch = psi_train_input[batch:batch + batch_size,:]
        label_batch = psi_train_label[batch:batch + batch_size,:]

        optimizer.zero_grad()

        output, mu, logvar = model(label_batch.float().cuda(), input_batch.float().cuda())

        # Reconstruction loss
        reconstruction_loss = F.mse_loss(output, label_batch.float().cuda(), reduction="sum")
        # KL divergence loss
        kl_loss = 0.5 * (mu ** 2 + torch.exp(logvar) - 1 - logvar).sum()
        
        # Total loss
        loss = reconstruction_loss + 10 * kl_loss

        loss.backward()
        optimizer.step()

    print(f'''Epoch {epoch+1}, 
          Reconstrunction Loss: {reconstruction_loss},
          KL Divergence Loss: {kl_loss}''')

    # Validation Loss
    for batch in range (0, valN, batch_size):

        input_batch = psi_val_input[batch:batch + batch_size,:]
        label_batch = psi_val_label[batch:batch + batch_size,:]
        
        output, mu, logvar = model(label_batch.float().cuda(), input_batch.float().cuda())
        reconstruction_loss = F.mse_loss(output, label_batch.float().cuda(), reduction="sum")
        kl_loss = 0.5 * (mu ** 2 + torch.exp(logvar) - 1 - logvar).sum()
        val_loss = reconstruction_loss + 10 * kl_loss # Experiment HIGHER coefficients
        # Print both reconstruction_loss and kl_loss

    print(f'''
          Validation Reconstrunction Loss: {reconstruction_loss},
          Validation KL Divergence Loss: {kl_loss}''')

    torch.save(model.state_dict(), f"{folder}/checkpoint_{epoch+1}")
    print(f"Model weights saved to {folder} with point {epoch+1}.")
    
    # Inference

    initial_cond = torch.reshape(torch.tensor(psi[0,:]), [1, 75])
    time_step = 30000
    z = torch.zeros([1,latent_dim])
    num_ens = 1
    pred = np.zeros ([time_step, 75, num_ens])


    for k in range (0, time_step):

        for ens in range (0, num_ens):
            if (k ==0):

                z = torch.randn_like(z)
                print(z.shape, initial_cond.shape)
                y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()
                pred[k,:,ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

            else:
                select_ens = np.random.randint(0,num_ens,1)
                z = torch.randn_like(z)
                y = (model.decode(z.float().cuda(),torch.reshape(torch.tensor(pred[k-1,:,select_ens]),[1,75]).float().cuda())).detach().cpu().numpy()
                pred[k,:, ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

    # Denormalize final preds
    print(std_psi[:, 63])
    pred_mean = pred * std_psi[:, 63] + mean_psi[:, 63]

    # Denormalize test labels
    actual_values = psi_train_label[:time_step, 63] * std_psi[:, 63] + mean_psi[:, 63]
    print(actual_values)

    plt.plot(pred_mean[0:30000,63],'r')
    plt.plot(actual_values[0:30000],'b')
    plt.title(f"Predictions vs Actual | Batch size of {batch_size}")
    plt.savefig(f'{folder}/prediction_epoch_{epoch+1}.png')
    plt.show()
    plt.cla()

torch.save(model.state_dict(), f"{model_weights_path}")
print(f"Model weights saved to {folder} with point {epoch+1}.")

In [ ]:
# Inference
import datetime
import os

# Initialize the model, optimizer, and loss function
model = ConditionalVAE(input_dim, latent_dim, output_dim, condition_dim)
model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# MODIFY THIS LINE FOR MODEL TESTING
model_weights_path = "/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Finetuned_Resnet_VAE_model_at_2025-03-29 16:48:56.932248/checkpoint_5"

if os.path.exists(model_weights_path):
    model.load_state_dict(torch.load(model_weights_path))
    print(f"Model weights loaded from {model_weights_path}.")

initial_cond = torch.reshape(torch.tensor(psi[0,:]), [1, 75])
time_step = 300000
z = torch.zeros([1,latent_dim])
num_ens = 1
pred = np.zeros ([time_step, 75, num_ens])

for k in range (0, time_step):

    for ens in range (0, num_ens):
        if (k ==0):

            z = torch.randn_like(z)
            print(z.shape, initial_cond.shape)
            y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()
            pred[k,:,ens] = y
            y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
            initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

        else:
            select_ens = np.random.randint(0,num_ens,1)
            z = torch.randn_like(z)
            y = (model.decode(z.float().cuda(),torch.reshape(torch.tensor(pred[k-1,:,select_ens]),[1,75]).float().cuda())).detach().cpu().numpy()
            pred[k,:, ens] = y
            y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
            initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

# Denormalize final preds
print(std_psi[:, 63])
pred_mean = pred * std_psi[:, 63] + mean_psi[:, 63]

# Denormalize test labels
actual_values = psi[:time_step, 63] * std_psi[:, 63] + mean_psi[:, 63]
print(actual_values)

plt.figure(figsize=(20,8))
plt.plot(pred_mean[0:300000,63],'r')
plt.plot(actual_values[0:300000],'b')
plt.grid(True)
plt.title(f"Predictions vs Actual | Checkpoint 4(best)")
plt.savefig(f'/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/plots/temp_preds/prediction_vs_actual_{datetime.datetime.now()}.png')
plt.show()
plt.cla()

# MODIFY THIS LINE FOR MODEL TESTING
np.save(f'/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Finetuned_Resnet_VAE_model_at_2025-03-29 16:48:56.932248/predictions_best_checkpoint_Finetuned_Resnet_VAE_1.npy', pred_mean)

In [ ]:
# Denormalize final preds
print(std_psi[:, 63])
pred_mean = pred * std_psi[:, 63] + mean_psi[:, 63]

# Denormalize test labels
actual_values = psi[:time_step, 63] * std_psi[:, 63] + mean_psi[:, 63]
print(actual_values)

plt.figure(figsize=(20,8))
plt.plot(pred_mean[0:300000,63],'r')
plt.plot(actual_values[0:300000],'b')
plt.grid(True)
plt.title(f"Predictions vs Actual | Checkpoint 5")
plt.savefig(f'{folder}/prediction_vs_actual.png')
plt.show()
plt.cla()

# Testing the Predictive Model

## Overview
This section outlines our testing strategy to validate the accuracy, robustness, and reliability of our predictive model. Rigorous testing is critical to ensure that our model performs well under various tests.
Set flags to 1 based on which metric(s) you would like to test.

In the next cell are the flags and some basic parameters.

In [ ]:
#FLAGS to determine testing
plot_data = 1
#what level do you want to plot
level = 63
CCDF = 1
Bi_modal_distribution = 1
single_step_profiles = 1
#for the single_step_profiles
NUM_SAMPLES = 5
#what weights do you want to use?
MODEL_PATH = r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_at_2025-03-25 16:07:01.635719/checkpoint_4_promising"
LEVEL = 63

There are two paths here that you will manually set. One to the physical data and the other to the predictions. 
# WARNING
Unless you're running new tests...
Do not run this cell multiple times after testing as it will create a new folder based on your current time!

In [ ]:
#Initialize Data
import numpy as np
import matplotlib.pyplot as plt
import datetime
import os

# Load the data; shape = (300000, 2, 75)
real_data = np.load(r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/data/actual/long_run_310k.npy")
predictions = np.load(r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Finetuned_Resnet_VAE_model_at_2025-03-29 16:48:56.932248/predictions_best_checkpoint_Finetuned_Resnet_VAE_1.npy")

#reshape the predictions so that it matches the real_data shape
predictions = predictions.reshape(300000, 1, 75)
print(predictions.shape)
print(real_data.shape)
timestamp = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S.%f")
folder = f"testing_at_{timestamp}"
os.mkdir(folder)
subfolders = ['timeseries', 'CCDF', 'bi_modal_distribution', 'single_step_profiles']
# Create each subdirectory inside the main folder
for subfolder in subfolders:
    path = os.path.join(folder, subfolder)
    os.mkdir(path)
    print(f"Created subfolder: {path}")
SAVE_DIR = os.path.join(folder, "single_step_profiles")

In [ ]:
# CCDF Error bars

import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import linregress
import seaborn as sns

CCDF_sum = 0

import datetime
import os

# Initialize the model, optimizer, and loss function
model = ConditionalVAE(input_dim, latent_dim, output_dim, condition_dim)
model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# MODIFY THIS LINE FOR MODEL TESTING
model_weights_path = "/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_at_2025-03-25 16:07:01.635719/checkpoint_4_promising"

if os.path.exists(model_weights_path):
    model.load_state_dict(torch.load(model_weights_path))
    print(f"Model weights loaded from {model_weights_path}.")
    
for i in range(0, 10):
    # Inference
    initial_cond = torch.reshape(torch.tensor(psi[0,:]), [1, 75])
    time_step = 300000
    z = torch.zeros([1,latent_dim])
    num_ens = 1
    pred = np.zeros ([time_step, 75, num_ens])

    for k in range (0, time_step):

        for ens in range (0, num_ens):
            if (k ==0):

                z = torch.randn_like(z)
                print(z.shape, initial_cond.shape)
                y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()
                pred[k,:,ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

            else:
                select_ens = np.random.randint(0,num_ens,1)
                z = torch.randn_like(z)
                y = (model.decode(z.float().cuda(),torch.reshape(torch.tensor(pred[k-1,:,select_ens]),[1,75]).float().cuda())).detach().cpu().numpy()
                pred[k,:, ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

    # Denormalize final preds
    print(std_psi[:, 63])
    pred_mean = pred * std_psi[:, 63] + mean_psi[:, 63]

    # Denormalize test labels
    actual_values = psi[:time_step, 63] * std_psi[:, 63] + mean_psi[:, 63]
    print(actual_values)

    # Load the data; shape = (300000, 2, 75)
    real_data = np.load(r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/data/actual/long_run_310k.npy")
    predictions = pred_mean.reshape(300000, 1, 75)

    if (CCDF):
        real_data_1d = real_data[:, 1, 63]  # Now shape is (309700,)
        predictions_1d = predictions[:, 0, 63]  # shape (300000,)

        # Define bounds (assuming they apply to both datasets)
        upper_bound = 53.8 / 2.8935
        lower_bound = 1.75 / 2.8935

        # Function to calculate transition durations
        def calculate_transition_durations(y_values, upper_bound, lower_bound):
            times_between_transitions = []
            transition_start = None
            above_upper = False
            below_lower = False

            for i in range(1, len(y_values)):
                if y_values[i] < lower_bound:  
                    below_lower = True
                    above_upper = False
                elif y_values[i] > upper_bound:  
                    if below_lower and transition_start is not None:
                        times_between_transitions.append(i - transition_start)
                        transition_start = None  
                    above_upper = True
                    below_lower = False

                if below_lower and transition_start is None:
                    transition_start = i

            return times_between_transitions

        # Compute transition durations for real data
        real_durations = calculate_transition_durations(real_data_1d, upper_bound, lower_bound)

        # Compute transition durations for predictions data
        pred_durations = calculate_transition_durations(predictions_1d, upper_bound, lower_bound)

        # Plot setup
        plt.figure(figsize=(10, 6))

        def bootstrap_ci(data, num_bootstraps=1000, confidence_level=0.95):
            sample_size = len(data)
            bootstrap_means = np.zeros(num_bootstraps)
            
            for i in range(num_bootstraps):
                bootstrap_sample = np.random.choice(data, size=sample_size, replace=True)
                bootstrap_means[i] = np.mean(bootstrap_sample)
            
            ci_lower = np.percentile(bootstrap_means, (1 - confidence_level) / 2 * 100)
            ci_upper = np.percentile(bootstrap_means, (1 + confidence_level) / 2 * 100)
            
            return np.mean(data), ci_lower, ci_upper

        # === REAL DATA CCDF AND FIT ===
        if len(real_durations) == 0:
            print("No transitions detected in real data with current bounds!")
        else:
            real_data_sorted = np.sort(real_durations)
            ccdf_real = 1 - np.arange(1, len(real_data_sorted) + 1) / len(real_data_sorted)

            valid_indices_real = ccdf_real > 0
            x_fit_real = real_data_sorted[valid_indices_real]
            y_fit_real = np.log(ccdf_real[valid_indices_real])

            slope_real, intercept_real, *_ = linregress(x_fit_real, y_fit_real)

            x_line_real = np.linspace(min(x_fit_real), max(x_fit_real), 40)
            y_line_real = np.exp(slope_real * x_line_real + intercept_real)

            # Create a grid of x values (time steps / durations)

            bootstrap_mean = []
            ci_lower_vals = []
            ci_upper_vals = []

            for x in x_line_real:
                valid_indices = (real_durations > x).astype(float)
                mean_val, lower, upper = bootstrap_ci(valid_indices)
                bootstrap_mean.append(mean_val)
                ci_lower_vals.append(lower)
                ci_upper_vals.append(upper)

            bootstrap_mean = np.array(bootstrap_mean)
            ci_lower_vals = np.array(ci_lower_vals)
            ci_upper_vals = np.array(ci_upper_vals)

            # Calculate error bars (difference from the bootstrap mean)
            error_lower = bootstrap_mean - ci_lower_vals
            error_upper = ci_upper_vals - bootstrap_mean

            plt.step(real_data_sorted, ccdf_real, where='post', label='Real Data CCDF', linewidth=2, linestyle='--')
            plt.errorbar(x_line_real, bootstrap_mean, yerr=[error_lower, error_upper],
                    fmt='o', color='blue', capsize=3, ecolor='blue', label='Real Bootstrap 95% CI')
            plt.plot(x_line_real, y_line_real, 'b-', label=f'Real Exp Fit (slope={slope_real:.4f})', linewidth=2)

        # === PREDICTIONS CCDF AND FIT ===
        if len(pred_durations) == 0:
            print("No transitions detected in predictions with current bounds!")
        else:
            pred_data_sorted = np.sort(pred_durations)
            ccdf_pred = 1 - np.arange(1, len(pred_data_sorted) + 1) / len(pred_data_sorted)

            valid_indices_pred = ccdf_pred > 0
            x_fit_pred = pred_data_sorted[valid_indices_pred]
            y_fit_pred = np.log(ccdf_pred[valid_indices_pred])

            slope_pred, intercept_pred, *_ = linregress(x_fit_pred, y_fit_pred)

            x_line_pred = np.linspace(min(x_fit_pred), max(x_fit_pred), 40)
            y_line_pred = np.exp(slope_pred * x_line_pred + intercept_pred)

            # Create a grid of x values (time steps / durations)

            bootstrap_mean = []
            ci_lower_vals = []
            ci_upper_vals = []

            for x in x_line_pred:
                valid_indices = (pred_durations > x).astype(float)
                mean_val, lower, upper = bootstrap_ci(valid_indices)
                bootstrap_mean.append(mean_val)
                ci_lower_vals.append(lower)
                ci_upper_vals.append(upper)

            bootstrap_mean = np.array(bootstrap_mean)
            ci_lower_vals = np.array(ci_lower_vals)
            ci_upper_vals = np.array(ci_upper_vals)

            # Calculate error bars (difference from the bootstrap mean)
            error_lower = bootstrap_mean - ci_lower_vals
            error_upper = ci_upper_vals - bootstrap_mean

            plt.step(pred_data_sorted, ccdf_pred, where='post', label='Predictions CCDF', linewidth=2, linestyle='-.', color='red')
            plt.errorbar(x_line_pred, bootstrap_mean, yerr=[error_lower, error_upper],
                    fmt='o', color='red', capsize=3, ecolor='red', label='Pred Bootstrap 95% CI')
            plt.plot(x_line_pred, y_line_pred, 'r-', label=f'Pred Exp Fit (slope={slope_pred:.4f})', linewidth=2)

            CCDF_sum += slope_pred

        # Plot labels and formatting
        plt.xlabel('Time Duration (Steps)')
        plt.ylabel('CCDF')
        plt.title('CCDF of Time Between B->A and A->B Transitions (Exponential Fit)')
        plt.yscale("log")  # y-axis log scale
        plt.xscale("linear")  # x-axis linear scale
        plt.grid()
        plt.legend()
        plt.tight_layout()
        save_path = os.path.join(folder, "CCDF")
        save_path = os.path.join(save_path, f"CCDF_plot{i}")
        plt.savefig(save_path)
        plt.show()

CCDF_avg = CCDF_sum/10
print(CCDF_avg)

In [ ]:
# Tests package

import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import linregress
import seaborn as sns
import random
if (plot_data):
    #note that the value 300000 will have to change depending on the real and predictions data length
    u_profile_real = real_data[:300000, 1, level]  # Match time length with predictions
    u_profile_pred = predictions[:, 0, level]
    time_steps = np.arange(len(u_profile_pred))

    # === Plot ===
    plt.figure(figsize=(14, 6))

    plt.plot(time_steps, u_profile_real, label='Real Data', alpha=0.7)
    plt.plot(time_steps, u_profile_pred, label='Predictions', linestyle='--')


    # Labels, legend, and formatting
    plt.xlabel('Time step')
    plt.ylabel('U (m/s)')
    plt.title(f'Time Series of U at Vertical Level {level}')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    save_path = os.path.join(folder, "timeseries")
    save_path = os.path.join(save_path, "real_prediction_plot")
    plt.savefig(save_path)
    plt.show()

if (CCDF):
    real_data_1d = real_data[:, 1, 61]  # Now shape is (309700,)
    predictions_1d = predictions[:, 0, 61]  # shape (300000,)

    # Define bounds (assuming they apply to both datasets)
    upper_bound = 53.8 / 2.8935
    lower_bound = 1.75 / 2.8935

    # Function to calculate transition durations
    def calculate_transition_durations(y_values, upper_bound, lower_bound):
        times_between_transitions = []
        transition_start = None
        above_upper = False
        below_lower = False

        for i in range(1, len(y_values)):
            if y_values[i] < lower_bound:  
                below_lower = True
                above_upper = False
            elif y_values[i] > upper_bound:  
                if below_lower and transition_start is not None:
                    times_between_transitions.append(i - transition_start)
                    transition_start = None  
                above_upper = True
                below_lower = False

            if below_lower and transition_start is None:
                transition_start = i

        return times_between_transitions

    # Compute transition durations for real data
    real_durations = calculate_transition_durations(real_data_1d, upper_bound, lower_bound)

    # Compute transition durations for predictions data
    pred_durations = calculate_transition_durations(predictions_1d, upper_bound, lower_bound)

    # Plot setup
    plt.figure(figsize=(10, 6))

    # === REAL DATA CCDF AND FIT ===
    if len(real_durations) == 0:
        print("No transitions detected in real data with current bounds!")
    else:
        real_data_sorted = np.sort(real_durations)
        ccdf_real = 1 - np.arange(1, len(real_data_sorted) + 1) / len(real_data_sorted)

        valid_indices_real = ccdf_real > 0
        x_fit_real = real_data_sorted[valid_indices_real]
        y_fit_real = np.log(ccdf_real[valid_indices_real])

        slope_real, intercept_real, *_ = linregress(x_fit_real, y_fit_real)

        x_line_real = np.linspace(min(x_fit_real), max(x_fit_real), 100)
        y_line_real = np.exp(slope_real * x_line_real + intercept_real)

        plt.step(real_data_sorted, ccdf_real, where='post', label='Real Data CCDF', linewidth=2, linestyle='--')
        plt.plot(x_line_real, y_line_real, 'b-', label=f'Real Exp Fit (slope={slope_real:.4f})', linewidth=2)

    # === PREDICTIONS CCDF AND FIT ===
    if len(pred_durations) == 0:
        print("No transitions detected in predictions with current bounds!")
    else:
        pred_data_sorted = np.sort(pred_durations)
        ccdf_pred = 1 - np.arange(1, len(pred_data_sorted) + 1) / len(pred_data_sorted)

        valid_indices_pred = ccdf_pred > 0
        x_fit_pred = pred_data_sorted[valid_indices_pred]
        y_fit_pred = np.log(ccdf_pred[valid_indices_pred])

        slope_pred, intercept_pred, *_ = linregress(x_fit_pred, y_fit_pred)

        x_line_pred = np.linspace(min(x_fit_pred), max(x_fit_pred), 100)
        y_line_pred = np.exp(slope_pred * x_line_pred + intercept_pred)

        plt.step(pred_data_sorted, ccdf_pred, where='post', label='Predictions CCDF', linewidth=2, linestyle='-.', color='red')
        plt.plot(x_line_pred, y_line_pred, 'r-', label=f'Pred Exp Fit (slope={slope_pred:.4f})', linewidth=2)

    # Plot labels and formatting
    plt.xlabel('Time Duration (Steps)')
    plt.ylabel('CCDF')
    plt.title('CCDF of Time Between B->A and A->B Transitions (Exponential Fit)')
    plt.yscale("log")  # y-axis log scale
    plt.xscale("linear")  # x-axis linear scale
    plt.grid()
    plt.legend()
    plt.tight_layout()
    save_path = os.path.join(folder, "CCDF")
    save_path = os.path.join(save_path, "CCDF_plot")
    plt.savefig(save_path)
    plt.show()
if (Bi_modal_distribution):
    zonal_wind_data_real = real_data[:, 1, 63]  # variable index 1 (e.g., zonal wind), level 60
    zonal_wind_data_predictions = predictions[:, 0, 63]  # variable index 0 (predictions), level 60

    print(f"Shape of zonal_wind_data_real: {zonal_wind_data_real.shape}")
    print(f"Shape of zonal_wind_data_predictions: {zonal_wind_data_predictions.shape}")

    # Plot the bimodal histogram
    plt.figure(figsize=(12, 6))

    # Create histograms (overlaid)
    sns.histplot(zonal_wind_data_real, bins=50, kde=True, color='black', alpha=0.6, element='step', label='Real Data')
    sns.histplot(zonal_wind_data_predictions, bins=50, kde=True, color='red', alpha=0.6, element='step', label='Predictions')

    # Customize plot labels and title
    plt.title('Distribution of Zonal Winds For Real Data and Predictions', fontsize=16)
    plt.xlabel('Zonal Wind (m/s)', fontsize=14)
    plt.ylabel('Frequency', fontsize=14)

    # Add vertical lines at means
    plt.axvline(np.mean(zonal_wind_data_real), color='black', linestyle='--', label=f'Real Mean: {np.mean(zonal_wind_data_real):.2f}')
    plt.axvline(np.mean(zonal_wind_data_predictions), color='red', linestyle='--', label=f'Pred Mean: {np.mean(zonal_wind_data_predictions):.2f}')

    # Final plot settings
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    save_path = os.path.join(folder, "bi_modal_distribution")
    save_path = os.path.join(save_path, "bi_modal_distribution_plot")
    plt.savefig(save_path)
    plt.show()
if (single_step_profiles):
    # Ensure save directory exists
    os.makedirs(SAVE_DIR, exist_ok=True)

    # === Load model weights ===
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()

    # === Randomly sample time points from real data ===
    time_indices = random.sample(range(0, real_data.shape[0] - 2), NUM_SAMPLES)
    print(f"Randomly sampled time steps: {time_indices}")

    # === Time series visualization ===
    real_data_timeseries = real_data[:, 1, LEVEL]
    time_steps_all = np.arange(len(real_data_timeseries))

    plt.figure(figsize=(12, 6))
    plt.plot(time_steps_all, real_data_timeseries, label="Real Data at Level 61", color='blue')

    # Mark sample points
    for idx_num, idx in enumerate(time_indices):
        plt.axvline(x=idx, color='green', linestyle='--', linewidth=2)
    if len(time_indices) > 0:
        plt.axvline(x=time_indices[0], color='green', linestyle='--', linewidth=2, label='Sampled Points')

    plt.title("Real Data Time Series with Sampled Points Highlighted")
    plt.xlabel("Time Step")
    plt.ylabel("U (m/s) at Level 61")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    save_path = os.path.join(SAVE_DIR, "real_data_timeseries_with_samples.png")
    plt.savefig(save_path)
    plt.show()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    # === Iterate over each sampled time point ===
    for i, time_step in enumerate(time_indices):
        next_time_step = time_step + 1

        # === Real data: current and next ===
        real_current = real_data[time_step, 1, :]       
        real_next = real_data[next_time_step, 1, :]      

        # === Normalize real_current and make prediction for next step ===
        initial_condition_normalized = (real_current.reshape(1, 75, 1) - mean_psi.reshape(1, -1, 1)) / std_psi.reshape(1, -1, 1)
        current_input = torch.tensor(initial_condition_normalized, dtype=torch.float32).to(device)

        with torch.no_grad():
            z = torch.randn_like(z)
            print(z.shape, initial_cond.shape)
            y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()

        # === Denormalize predicted next ===
        pred_next_denorm = y.squeeze() * std_psi.squeeze() + mean_psi.squeeze()

        # === Extract U, Re(Psi), Im(Psi) components ===
        # U profiles
        U_current_real = real_current[51:74]
        U_next_real = real_next[51:74]
        U_next_pred = pred_next_denorm[51:74]

        # Re(Psi) profiles
        RePsi_current_real = real_current[0:24]
        RePsi_next_real = real_next[0:24]
        RePsi_next_pred = pred_next_denorm[0:24]

        # Im(Psi) profiles
        ImPsi_current_real = real_current[25:50]
        ImPsi_next_real = real_next[25:50]
        ImPsi_next_pred = pred_next_denorm[25:50]

        # === Differences ===
        U_diff_real = U_next_real - U_current_real
        U_diff_pred = U_next_pred - U_current_real

        RePsi_diff_real = RePsi_next_real - RePsi_current_real
        RePsi_diff_pred = RePsi_next_pred - RePsi_current_real

        ImPsi_diff_real = ImPsi_next_real - ImPsi_current_real
        ImPsi_diff_pred = ImPsi_next_pred - ImPsi_current_real

        # === Create a single figure with 3 rows (U, Re(Psi), Im(Psi)) ===
        fig, axes = plt.subplots(3, 2, figsize=(16, 18))  # 3 rows, 2 columns (Profile and Difference)

        z_levels_U = np.linspace(0, 70, 23)
        z_levels_RePsi = np.linspace(0, 70, 24)
        z_levels_ImPsi = np.linspace(0, 70, 25)

        # --- U ---
        axes[0, 0].plot(U_current_real, z_levels_U, 'x-', label="Real Current")
        axes[0, 0].plot(U_next_real, z_levels_U, 'd-', label="Real Next")
        axes[0, 0].plot(U_next_pred, z_levels_U, 's--', label="Predicted Next")
        axes[0, 0].set_title(f"U Profiles @ Step {time_step}")
        axes[0, 0].set_xlabel("U (m/s)")
        axes[0, 0].set_ylabel("Vertical Levels (km)")
        axes[0, 0].legend()

        axes[0, 1].plot(U_diff_real, z_levels_U, 'xb', label="Real Δ (Next - Current)")
        axes[0, 1].plot(U_diff_pred, z_levels_U, 'o--r', label="Pred Δ (Next - Current)")
        axes[0, 1].set_title("U Difference (Next - Current)")
        axes[0, 1].set_xlabel("ΔU (m/s)")
        axes[0, 1].legend()

        # --- Re(Psi) ---
        axes[1, 0].plot(RePsi_current_real, z_levels_RePsi, 'x-', label="Real Current")
        axes[1, 0].plot(RePsi_next_real, z_levels_RePsi, 'd-', label="Real Next")
        axes[1, 0].plot(RePsi_next_pred, z_levels_RePsi, 's--', label="Predicted Next")
        axes[1, 0].set_title(f"Re(Psi) Profiles @ Step {time_step}")
        axes[1, 0].set_xlabel("Re(Psi)")
        axes[1, 0].set_ylabel("Vertical Levels (km)")
        axes[1, 0].legend()

        axes[1, 1].plot(RePsi_diff_real, z_levels_RePsi, 'xb', label="Real Δ (Next - Current)")
        axes[1, 1].plot(RePsi_diff_pred, z_levels_RePsi, 'o--r', label="Pred Δ (Next - Current)")
        axes[1, 1].set_title("Re(Psi) Difference (Next - Current)")
        axes[1, 1].set_xlabel("ΔRe(Psi)")
        axes[1, 1].legend()

        # --- Im(Psi) ---
        axes[2, 0].plot(ImPsi_current_real, z_levels_ImPsi, 'x-', label="Real Current")
        axes[2, 0].plot(ImPsi_next_real, z_levels_ImPsi, 'd-', label="Real Next")
        axes[2, 0].plot(ImPsi_next_pred, z_levels_ImPsi, 's--', label="Predicted Next")
        axes[2, 0].set_title(f"Im(Psi) Profiles @ Step {time_step}")
        axes[2, 0].set_xlabel("Im(Psi)")
        axes[2, 0].set_ylabel("Vertical Levels (km)")
        axes[2, 0].legend()

        axes[2, 1].plot(ImPsi_diff_real, z_levels_ImPsi, 'xb', label="Real Δ (Next - Current)")
        axes[2, 1].plot(ImPsi_diff_pred, z_levels_ImPsi, 'o--r', label="Pred Δ (Next - Current)")
        axes[2, 1].set_title("Im(Psi) Difference (Next - Current)")
        axes[2, 1].set_xlabel("ΔIm(Psi)")
        axes[2, 1].legend()

        # === Finalize and Save ===
        plt.suptitle(f"Single Step Profile Comparisons at Time Step {time_step}", fontsize=18)
        plt.tight_layout(rect=[0, 0.03, 1, 0.97])

        save_path = os.path.join(SAVE_DIR, f"Profile_Summary_point_{time_step}.png")
        plt.savefig(save_path)
        plt.show()

        print(f"Combined plot saved for sampled point {time_step}")

    # Final debug
    print("Finished processing all sampled points.")
        # Debugging prints
    print(predictions.shape) 
    print(real_data.shape)

# Climate Loss Training

In [ ]:
# Total Variation Distance
def total_variation_distance(p,q):
	p = np.array(p)
	q = np.array(q)
	return 0.5 * np.sum(np.abs(p-q))

predictions = np.load(r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/meetings/March_28th/ResNet/Inferences/predictions_best_checkpoint_Resnet_VAE_4.npy")
real_data = np.load(r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/data/actual/long_run_310k.npy")
print(predictions.shape)
#actual implementation
pred_hist, _ = np.histogram(predictions, bins=50, density=True)
actual_hist, _ = np.histogram(real_data,bins=50,density=True)

tvd = total_variation_distance(pred_hist,actual_hist)
print(f"TVD: {tvd}")

In [ ]:
from scipy.stats import linregress
import os

# Initialize the model, optimizer, and loss function
model = ConditionalVAE(input_dim, latent_dim, output_dim, condition_dim)
model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

model_weights_path = "/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_at_2025-03-25 16:07:01.635719/checkpoint_4_promising"

if os.path.exists(model_weights_path):
    model.load_state_dict(torch.load(model_weights_path))
    print(f"Model weights loaded from {model_weights_path}.")

def climate_loss_function(psi, model, latent_dim=1024, time_step=batch_size, num_ens=1):
    if (CCDF):
            # Inference
            initial_cond = torch.reshape(torch.tensor(psi[0,:]), [1, 75])
            z = torch.zeros([1,latent_dim])
            num_ens = 1
            pred = np.zeros ([time_step, 75, num_ens])

            for k in range (0, time_step):

                for ens in range (0, num_ens):
                    if (k ==0):

                        z = torch.randn_like(z)
                        print(z.shape, initial_cond.shape)
                        y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()
                        pred[k,:,ens] = y
                        y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                        initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

                    else:
                        select_ens = np.random.randint(0,num_ens,1)
                        z = torch.randn_like(z)
                        y = (model.decode(z.float().cuda(),torch.reshape(torch.tensor(pred[k-1,:,select_ens]),[1,75]).float().cuda())).detach().cpu().numpy()
                        pred[k,:, ens] = y
                        y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                        initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

            # Denormalize final preds
            print(std_psi[:, 63])
            pred_mean = pred * std_psi[:, 63] + mean_psi[:, 63]

            # Denormalize test labels
            actual_values = psi[:time_step, 63] * std_psi[:, 63] + mean_psi[:, 63]
            print(actual_values)

            # Load the data; shape = (300000, 2, 75)
            real_data = np.load(r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/data/actual/long_run_310k.npy")
            predictions = pred_mean.reshape(time_step, 1, 75)

            real_data_1d = real_data[:, 0, 63]  # Now shape is (309700,)
            predictions_1d = predictions[:, 0, 63]  # shape (300000,)

            # Define bounds (assuming they apply to both datasets)
            upper_bound = 53.8 / 2.8935
            lower_bound = 1.75 / 2.8935

            # Function to calculate transition durations
            def calculate_transition_durations(y_values, upper_bound, lower_bound):
                times_between_transitions = []
                transition_start = None
                above_upper = False
                below_lower = False

                for i in range(1, len(y_values)):
                    if y_values[i] < lower_bound:  
                        below_lower = True
                        above_upper = False
                    elif y_values[i] > upper_bound:  
                        if below_lower and transition_start is not None:
                            times_between_transitions.append(i - transition_start)
                            transition_start = None  
                        above_upper = True
                        below_lower = False

                    if below_lower and transition_start is None:
                        transition_start = i

                return times_between_transitions

            # Compute transition durations for real data
            real_durations = calculate_transition_durations(real_data_1d, upper_bound, lower_bound)

            # Compute transition durations for predictions data
            pred_durations = calculate_transition_durations(predictions_1d, upper_bound, lower_bound)

            def bootstrap_ci(data, num_bootstraps=1000, confidence_level=0.95):
                sample_size = len(data)
                bootstrap_means = np.zeros(num_bootstraps)
                
                for i in range(num_bootstraps):
                    bootstrap_sample = np.random.choice(data, size=sample_size, replace=True)
                    bootstrap_means[i] = np.mean(bootstrap_sample)
                
                ci_lower = np.percentile(bootstrap_means, (1 - confidence_level) / 2 * 100)
                ci_upper = np.percentile(bootstrap_means, (1 + confidence_level) / 2 * 100)
                
                return np.mean(data), ci_lower, ci_upper

            # === REAL DATA CCDF AND FIT ===
            if len(real_durations) == 0:
                print("No transitions detected in real data with current bounds!")
            else:
                real_data_sorted = np.sort(real_durations)
                ccdf_real = 1 - np.arange(1, len(real_data_sorted) + 1) / len(real_data_sorted)

                valid_indices_real = ccdf_real > 0
                x_fit_real = real_data_sorted[valid_indices_real]
                y_fit_real = np.log(ccdf_real[valid_indices_real])

                slope_real, intercept_real, *_ = linregress(x_fit_real, y_fit_real)

                x_line_real = np.linspace(min(x_fit_real), max(x_fit_real), 40)
                y_line_real = np.exp(slope_real * x_line_real + intercept_real)

                # Create a grid of x values (time steps / durations)

                bootstrap_mean = []
                ci_lower_vals = []
                ci_upper_vals = []

                for x in x_line_real:
                    valid_indices = (real_durations > x).astype(float)
                    mean_val, lower, upper = bootstrap_ci(valid_indices)
                    bootstrap_mean.append(mean_val)
                    ci_lower_vals.append(lower)
                    ci_upper_vals.append(upper)

                bootstrap_mean = np.array(bootstrap_mean)
                ci_lower_vals = np.array(ci_lower_vals)
                ci_upper_vals = np.array(ci_upper_vals)

                # Calculate error bars (difference from the bootstrap mean)
                error_lower = bootstrap_mean - ci_lower_vals
                error_upper = ci_upper_vals - bootstrap_mean

                plt.step(real_data_sorted, ccdf_real, where='post', label='Real Data CCDF', linewidth=2, linestyle='--')
                plt.errorbar(x_line_real, bootstrap_mean, yerr=[error_lower, error_upper],
                        fmt='o', color='blue', capsize=3, ecolor='blue', label='Real Bootstrap 95% CI')
                plt.plot(x_line_real, y_line_real, 'b-', label=f'Real Exp Fit (slope={slope_real:.4f})', linewidth=2)

            # === PREDICTIONS CCDF AND FIT ===
            if len(pred_durations) == 0:
                print("No transitions detected in predictions with current bounds!")
            else:
                pred_data_sorted = np.sort(pred_durations)
                ccdf_pred = 1 - np.arange(1, len(pred_data_sorted) + 1) / len(pred_data_sorted)

                valid_indices_pred = ccdf_pred > 0
                x_fit_pred = pred_data_sorted[valid_indices_pred]
                y_fit_pred = np.log(ccdf_pred[valid_indices_pred])

                slope_pred, intercept_pred, *_ = linregress(x_fit_pred, y_fit_pred)

                x_line_pred = np.linspace(min(x_fit_pred), max(x_fit_pred), 40)
                y_line_pred = np.exp(slope_pred * x_line_pred + intercept_pred)

                # Create a grid of x values (time steps / durations)

                bootstrap_mean = []
                ci_lower_vals = []
                ci_upper_vals = []

                for x in x_line_pred:
                    valid_indices = (pred_durations > x).astype(float)
                    mean_val, lower, upper = bootstrap_ci(valid_indices)
                    bootstrap_mean.append(mean_val)
                    ci_lower_vals.append(lower)
                    ci_upper_vals.append(upper)

                bootstrap_mean = np.array(bootstrap_mean)
                ci_lower_vals = np.array(ci_lower_vals)
                ci_upper_vals = np.array(ci_upper_vals)

                # Calculate error bars (difference from the bootstrap mean)
                error_lower = bootstrap_mean - ci_lower_vals
                error_upper = ci_upper_vals - bootstrap_mean

                plt.step(pred_data_sorted, ccdf_pred, where='post', label='Predictions CCDF', linewidth=2, linestyle='-.', color='red')
                plt.errorbar(x_line_pred, bootstrap_mean, yerr=[error_lower, error_upper],
                        fmt='o', color='red', capsize=3, ecolor='red', label='Pred Bootstrap 95% CI')
                plt.plot(x_line_pred, y_line_pred, 'r-', label=f'Pred Exp Fit (slope={slope_pred:.4f})', linewidth=2)

                CCDF_loss = abs(slope_real-slope_pred)
            
            # Plot labels and formatting
            plt.xlabel('Time Duration (Steps)')
            plt.ylabel('CCDF')
            plt.title('CCDF of Time Between B->A and A->B Transitions (Exponential Fit)')
            plt.yscale("log")  # y-axis log scale
            plt.xscale("linear")  # x-axis linear scale
            plt.grid()
            plt.legend()
            plt.tight_layout()
            plt.show()

            pred_hist, _ = np.histogram(predictions, bins=100, density=True)
            actual_hist, _ = np.histogram(real_data,bins=100,density=True)

            tvd = total_variation_distance(pred_hist,actual_hist)

            return CCDF_loss, tvd

CCDF_loss, tvd = climate_loss_function(psi, model, 1024, 300000, 1)
climate_loss = tvd * 0.5 + CCDF_loss * 500
print(climate_loss)

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from scipy.stats import linregress

class ClimateModelLoss(nn.Module):
    def __init__(self):
        super(ClimateModelLoss, self).__init__()

    def calculate_transition_durations(self, y_values):
        upper_bound=53.8 / 2.8935 
        lower_bound=1.75 / 2.8935
        times_between_transitions = []
        transition_start = None
        above_upper = False
        below_lower = False

        for i in range(1, len(y_values)):
            if y_values[i] < lower_bound:  
                below_lower = True
                above_upper = False
            elif y_values[i] > upper_bound:  
                if below_lower and transition_start is not None:
                    times_between_transitions.append(i - transition_start)
                    transition_start = None  
                above_upper = True
                below_lower = False

            if below_lower and transition_start is None:
                transition_start = i

        return times_between_transitions
    
    def compute_ccdf_slope(self, durations):
        if len(durations) == 0:
            return 0  # Return 0 if no transitions detected
        
        # Sort durations and compute CCDF
        data_sorted = np.sort(durations)
        ccdf = 1 - np.arange(1, len(data_sorted) + 1) / len(data_sorted)
        
        # Prepare data for linear regression
        valid_indices = ccdf > 0
        x_fit = data_sorted[valid_indices]
        
        # Check if we have enough valid data points
        if len(x_fit) < 2:
            return 0  # Not enough points for regression
        
        # Check if all x values are identical
        if np.allclose(x_fit, x_fit[0], rtol=1e-10, atol=1e-10):
            # If all x values are the same, we can't compute a meaningful slope
            # Return a default value or handle the edge case appropriately
            return 0
    
        y_fit = np.log(ccdf[valid_indices])
        
        # Compute slope using linear regression
        slope, intercept, r_value, p_value, std_err = linregress(x_fit, y_fit)
        
        return slope
    
    def forward(self, predictions):
        
        # Select the specific column for analysis (column 63 in original code)
        # and denormalize data
        pred_denorm = predictions * std_psi[:, 63] + mean_psi[:, 63]
        pred_1d = pred_denorm[:, 0, 63]
        
        # Compute real data transition durations (from loaded reference data)
        real_data_1d = real_data[:, 0, 63]
        
        # Compute transition durations
        real_durations = self.calculate_transition_durations(real_data_1d)
        pred_durations = self.calculate_transition_durations(pred_1d)
        
        # Compute CCDF slopes
        real_slope = self.compute_ccdf_slope(real_durations)
        pred_slope = self.compute_ccdf_slope(pred_durations)
        
        # CCDF loss (difference in slopes)
        ccdf_loss = abs(real_slope - pred_slope)
        
        # Total Variation Distance
        pred_hist, _ = np.histogram(pred_1d, bins=100, density=True)
        real_hist, _ = np.histogram(real_data_1d, bins=100, density=True)
        tvd = np.sum(np.abs(pred_hist - real_hist)) / 2
        
        # Combine losses with weights
        total_loss = 0 * ccdf_loss + 20 * tvd
        
        return torch.tensor(total_loss, dtype=torch.float32, requires_grad=True)

In [ ]:
# Training

import datetime
import os

# Initialize the model, optimizer, and loss function
model = ConditionalVAE(input_dim, latent_dim, output_dim, condition_dim)
model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

folder = f"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Finetuned_Resnet_VAE_model_at_{datetime.datetime.now()}"
os.makedirs(folder)

# MODIFY THIS LINE FOR MODEL TESTING
model_weights_path = "/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_at_2025-03-25 16:07:01.635719/checkpoint_4_promising"

if os.path.exists(model_weights_path):
    model.load_state_dict(torch.load(model_weights_path))
    print(f"Model weights loaded from {model_weights_path}.")

num_epochs = 5
climate_loss_fn = ClimateModelLoss()

for epoch in range(num_epochs):
    
    model.train()
    optimizer.zero_grad()

    initial_cond = torch.reshape(torch.tensor(psi[0,:]), [1, 75])
    time_step = 0
    z = torch.zeros([1, latent_dim])
    num_ens = 1
    pred = np.zeros ([time_step, 75, num_ens])

    for k in range (0, time_step):

        for ens in range (0, num_ens):
            if (k ==0):

                z = torch.randn_like(z)
                print(z.shape, initial_cond.shape)
                y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()
                pred[k,:,ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

            else:
                select_ens = np.random.randint(0,num_ens,1)
                z = torch.randn_like(z)
                y = (model.decode(z.float().cuda(),torch.reshape(torch.tensor(pred[k-1,:,select_ens]),[1,75]).float().cuda())).detach().cpu().numpy()
                pred[k,:, ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

    # Denormalize final preds
    print(std_psi[:, 63])
    pred_mean = pred * std_psi[:, 63] + mean_psi[:, 63]

    # Denormalize test labels
    actual_values = psi[:time_step, 63] * std_psi[:, 63] + mean_psi[:, 63]
    print(actual_values)

    plt.figure(figsize=(20,8))
    plt.plot(pred_mean[0:300000,63],'r')
    plt.plot(actual_values[0:300000],'b')
    plt.grid(True)
    plt.title(f"Predictions vs Actual | Checkpoint 4(best)")
    plt.savefig(f'/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/plots/temp_preds/prediction_vs_actual_{datetime.datetime.now()}.png')
    plt.show()
    plt.cla()

    loss = climate_loss_fn(predictions)

    loss.backward()
    optimizer.step()

    print(f'''Epoch {epoch+1}, 
          Climate Loss: {loss}''')

    # Validation Loss
    initial_cond = torch.reshape(torch.tensor(psi[0,:]), [1, 75])
    time_step = 300000
    z = torch.zeros([1, latent_dim])
    num_ens = 1
    pred = np.zeros ([time_step, 75, num_ens])

    for k in range (0, time_step):

        for ens in range (0, num_ens):
            if (k ==0):

                z = torch.randn_like(z)
                print(z.shape, initial_cond.shape)
                y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()
                pred[k,:,ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

            else:
                select_ens = np.random.randint(0,num_ens,1)
                z = torch.randn_like(z)
                y = (model.decode(z.float().cuda(),torch.reshape(torch.tensor(pred[k-1,:,select_ens]),[1,75]).float().cuda())).detach().cpu().numpy()
                pred[k,:, ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

    # Denormalize final preds
    print(std_psi[:, 63])
    pred_mean = pred * std_psi[:, 63] + mean_psi[:, 63]

    # Denormalize test labels
    actual_values = psi[:time_step, 63] * std_psi[:, 63] + mean_psi[:, 63]
    print(actual_values)

    plt.figure(figsize=(20,8))
    plt.plot(pred_mean[0:300000,63],'r')
    plt.plot(actual_values[0:300000],'b')
    plt.grid(True)
    plt.title(f"Predictions vs Actual | Checkpoint 4(best)")
    plt.savefig(f'/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/plots/temp_preds/prediction_vs_actual_{datetime.datetime.now()}.png')
    plt.show()
    plt.cla()

    val_loss = climate_loss_fn(predictions)

    print(f'''Epoch {epoch+1}, 
          Validation Climate Loss: {val_loss}''')

    torch.save(model.state_dict(), f"{folder}/checkpoint_{epoch+1}")
    print(f"Model weights saved to {folder} with point {epoch+1}.")

# Inference

initial_cond = torch.reshape(torch.tensor(psi[0,:]), [1, 75])
time_step = 30000
z = torch.zeros([1,latent_dim])
num_ens = 1
pred = np.zeros ([time_step, 75, num_ens])

for k in range (0, time_step):

    for ens in range (0, num_ens):
        if (k ==0):

            z = torch.randn_like(z)
            print(z.shape, initial_cond.shape)
            y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()
            pred[k,:,ens] = y
            y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
            initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

        else:
            select_ens = np.random.randint(0,num_ens,1)
            z = torch.randn_like(z)
            y = (model.decode(z.float().cuda(),torch.reshape(torch.tensor(pred[k-1,:,select_ens]),[1,75]).float().cuda())).detach().cpu().numpy()
            pred[k,:, ens] = y
            y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
            initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

# Denormalize final preds
print(std_psi[:, 63])
pred_mean = pred * std_psi[:, 63] + mean_psi[:, 63]

# Denormalize test labels
actual_values = psi_train_label[:time_step, 63] * std_psi[:, 63] + mean_psi[:, 63]
print(actual_values)

plt.plot(pred_mean[0:30000,63],'r')
plt.plot(actual_values[0:30000],'b')
plt.title(f"Predictions vs Actual | Batch size of {batch_size}")
plt.savefig(f'{folder}/prediction_epoch_{epoch+1}.png')
plt.show()
plt.cla()

torch.save(model.state_dict(), f"{model_weights_path}")
print(f"Model weights saved to {folder} with point {epoch+1}.")

In [ ]:
# CCDF Error bars

import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import linregress
import seaborn as sns

CCDF_sum = 0

import datetime
import os

# Initialize the model, optimizer, and loss function
model = ConditionalVAE(input_dim, latent_dim, output_dim, condition_dim)
model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

# MODIFY THIS LINE FOR MODEL TESTING
model_weights_path = "/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Finetuned_Resnet_VAE_model_at_2025-03-29 16:09:38.411062/checkpoint_5"

if os.path.exists(model_weights_path):
    model.load_state_dict(torch.load(model_weights_path))
    print(f"Model weights loaded from {model_weights_path}.")
    
for i in range(0, 1):
    # Inference
    initial_cond = torch.reshape(torch.tensor(psi[0,:]), [1, 75])
    time_step = 300000
    z = torch.zeros([1,latent_dim])
    num_ens = 1
    pred = np.zeros ([time_step, 75, num_ens])

    for k in range (0, time_step):

        for ens in range (0, num_ens):
            if (k == 0):

                z = torch.randn_like(z)
                print(z.shape, initial_cond.shape)
                y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()
                pred[k,:,ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

            else:
                select_ens = np.random.randint(0,num_ens,1)
                z = torch.randn_like(z)
                y = (model.decode(z.float().cuda(),torch.reshape(torch.tensor(pred[k-1,:,select_ens]),[1,75]).float().cuda())).detach().cpu().numpy()
                pred[k,:, ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

    # Denormalize final preds
    print(std_psi[:, 63])
    pred_mean = pred * std_psi[:, 63] + mean_psi[:, 63]

    # Denormalize test labels
    actual_values = psi[:time_step, 63] * std_psi[:, 63] + mean_psi[:, 63]
    print(actual_values)

    # Load the data; shape = (300000, 2, 75)
    real_data = np.load(r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/data/actual/long_run_310k.npy")
    predictions = pred_mean.reshape(300000, 1, 75)

    if (CCDF):
        real_data_1d = real_data[:, 1, 63]  # Now shape is (309700,)
        predictions_1d = predictions[:, 0, 63]  # shape (300000,)

        # Define bounds (assuming they apply to both datasets)
        upper_bound = 53.8 / 2.8935
        lower_bound = 1.75 / 2.8935

        # Function to calculate transition durations
        def calculate_transition_durations(y_values, upper_bound, lower_bound):
            times_between_transitions = []
            transition_start = None
            above_upper = False
            below_lower = False

            for i in range(1, len(y_values)):
                if y_values[i] < lower_bound:  
                    below_lower = True
                    above_upper = False
                elif y_values[i] > upper_bound:  
                    if below_lower and transition_start is not None:
                        times_between_transitions.append(i - transition_start)
                        transition_start = None  
                    above_upper = True
                    below_lower = False

                if below_lower and transition_start is None:
                    transition_start = i

            return times_between_transitions

        # Compute transition durations for real data
        real_durations = calculate_transition_durations(real_data_1d, upper_bound, lower_bound)

        # Compute transition durations for predictions data
        pred_durations = calculate_transition_durations(predictions_1d, upper_bound, lower_bound)

        # Plot setup
        plt.figure(figsize=(10, 6))

        def bootstrap_ci(data, num_bootstraps=1000, confidence_level=0.95):
            sample_size = len(data)
            bootstrap_means = np.zeros(num_bootstraps)
            
            for i in range(num_bootstraps):
                bootstrap_sample = np.random.choice(data, size=sample_size, replace=True)
                bootstrap_means[i] = np.mean(bootstrap_sample)
            
            ci_lower = np.percentile(bootstrap_means, (1 - confidence_level) / 2 * 100)
            ci_upper = np.percentile(bootstrap_means, (1 + confidence_level) / 2 * 100)
            
            return np.mean(data), ci_lower, ci_upper

        # === REAL DATA CCDF AND FIT ===
        if len(real_durations) == 0:
            print("No transitions detected in real data with current bounds!")
        else:
            real_data_sorted = np.sort(real_durations)
            ccdf_real = 1 - np.arange(1, len(real_data_sorted) + 1) / len(real_data_sorted)

            valid_indices_real = ccdf_real > 0
            x_fit_real = real_data_sorted[valid_indices_real]
            y_fit_real = np.log(ccdf_real[valid_indices_real])

            slope_real, intercept_real, *_ = linregress(x_fit_real, y_fit_real)

            x_line_real = np.linspace(min(x_fit_real), max(x_fit_real), 40)
            y_line_real = np.exp(slope_real * x_line_real + intercept_real)

            # Create a grid of x values (time steps / durations)

            bootstrap_mean = []
            ci_lower_vals = []
            ci_upper_vals = []

            for x in x_line_real:
                valid_indices = (real_durations > x).astype(float)
                mean_val, lower, upper = bootstrap_ci(valid_indices)
                bootstrap_mean.append(mean_val)
                ci_lower_vals.append(lower)
                ci_upper_vals.append(upper)

            bootstrap_mean = np.array(bootstrap_mean)
            ci_lower_vals = np.array(ci_lower_vals)
            ci_upper_vals = np.array(ci_upper_vals)

            # Calculate error bars (difference from the bootstrap mean)
            error_lower = bootstrap_mean - ci_lower_vals
            error_upper = ci_upper_vals - bootstrap_mean

            plt.step(real_data_sorted, ccdf_real, where='post', label='Real Data CCDF', linewidth=2, linestyle='--')
            plt.errorbar(x_line_real, bootstrap_mean, yerr=[error_lower, error_upper],
                    fmt='o', color='blue', capsize=3, ecolor='blue', label='Real Bootstrap 95% CI')
            plt.plot(x_line_real, y_line_real, 'b-', label=f'Real Exp Fit (slope={slope_real:.4f})', linewidth=2)

        # === PREDICTIONS CCDF AND FIT ===
        if len(pred_durations) == 0:
            print("No transitions detected in predictions with current bounds!")
        else:
            pred_data_sorted = np.sort(pred_durations)
            ccdf_pred = 1 - np.arange(1, len(pred_data_sorted) + 1) / len(pred_data_sorted)

            valid_indices_pred = ccdf_pred > 0
            x_fit_pred = pred_data_sorted[valid_indices_pred]
            y_fit_pred = np.log(ccdf_pred[valid_indices_pred])

            slope_pred, intercept_pred, *_ = linregress(x_fit_pred, y_fit_pred)

            x_line_pred = np.linspace(min(x_fit_pred), max(x_fit_pred), 40)
            y_line_pred = np.exp(slope_pred * x_line_pred + intercept_pred)

            # Create a grid of x values (time steps / durations)

            bootstrap_mean = []
            ci_lower_vals = []
            ci_upper_vals = []

            for x in x_line_pred:
                valid_indices = (pred_durations > x).astype(float)
                mean_val, lower, upper = bootstrap_ci(valid_indices)
                bootstrap_mean.append(mean_val)
                ci_lower_vals.append(lower)
                ci_upper_vals.append(upper)

            bootstrap_mean = np.array(bootstrap_mean)
            ci_lower_vals = np.array(ci_lower_vals)
            ci_upper_vals = np.array(ci_upper_vals)

            # Calculate error bars (difference from the bootstrap mean)
            error_lower = bootstrap_mean - ci_lower_vals
            error_upper = ci_upper_vals - bootstrap_mean

            plt.step(pred_data_sorted, ccdf_pred, where='post', label='Predictions CCDF', linewidth=2, linestyle='-.', color='red')
            plt.errorbar(x_line_pred, bootstrap_mean, yerr=[error_lower, error_upper],
                    fmt='o', color='red', capsize=3, ecolor='red', label='Pred Bootstrap 95% CI')
            plt.plot(x_line_pred, y_line_pred, 'r-', label=f'Pred Exp Fit (slope={slope_pred:.4f})', linewidth=2)

            CCDF_sum += slope_pred

        # Plot labels and formatting
        plt.xlabel('Time Duration (Steps)')
        plt.ylabel('CCDF')
        plt.title('CCDF of Time Between B->A and A->B Transitions (Exponential Fit)')
        plt.yscale("log")  # y-axis log scale
        plt.xscale("linear")  # x-axis linear scale
        plt.grid()
        plt.legend()
        plt.tight_layout()
        save_path = os.path.join(folder, "CCDF")
        plt.savefig(save_path)
        plt.show()

CCDF_avg = CCDF_sum/10
print(CCDF_avg)

# EXPERIMENTAL

In [ ]:
import torch
import torch.nn as nn
import numpy as np
from scipy.stats import linregress

class ClimateModelLoss(nn.Module):
    def __init__(self):
        super(ClimateModelLoss, self).__init__()
    
    def forward(self, predictions, real_data):
        
        # Select the specific column for analysis (column 63 in original code)
        # and denormalize data
        pred_denorm = predictions.detach().cpu().numpy() * std_psi[:, 63] + mean_psi[:, 63]
        pred_1d = pred_denorm[:, 63]
        
        # Compute real data transition durations (from loaded reference data)
        real_data_1d = real_data[:, 63]
        
        # Total Variation Distance
        pred_hist, _ = np.histogram(pred_1d, bins=10, density=True)
        real_hist, _ = np.histogram(real_data_1d.detach().cpu().numpy(), bins=10, density=True)
        tvd = np.sum(np.abs(pred_hist - real_hist)) / 2
        
        # Combine losses with weights
        total_loss = tvd
        
        return torch.tensor(total_loss, dtype=torch.float32, requires_grad=True)

In [ ]:
# Training

import datetime
import os

# Initialize the model, optimizer, and loss function
model = ConditionalVAE(input_dim, latent_dim, output_dim, condition_dim)
model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

folder = f"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Finetuned_Resnet_VAE_model_at_{datetime.datetime.now()}"
os.makedirs(folder)

# MODIFY THIS LINE FOR MODEL TESTING
model_weights_path = "/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/training_cycles/resnet/Resnet_VAE_model_at_2025-03-25 16:07:01.635719/checkpoint_4_promising"

if os.path.exists(model_weights_path):
    model.load_state_dict(torch.load(model_weights_path))
    print(f"Model weights loaded from {model_weights_path}.")

num_epochs = 50
climate_loss_fn = ClimateModelLoss()

# Train the model
for epoch in range(num_epochs):
    model.train()
    for batch in range (0, trainN, batch_size):

        input_batch = psi_train_input[batch:batch + batch_size,:]
        label_batch = psi_train_label[batch:batch + batch_size,:]

        optimizer.zero_grad()

        output, mu, logvar = model(label_batch.float().cuda(), input_batch.float().cuda())

        loss = climate_loss_fn(output, label_batch.float().cuda())

        loss.backward()
        optimizer.step()

    print(f'''Epoch {epoch+1}, 
          TVD Loss: {loss}''')

    # Validation Loss
    for batch in range (0, valN, batch_size):

        input_batch = psi_val_input[batch:batch + batch_size,:]
        label_batch = psi_val_label[batch:batch + batch_size,:]
        
        output, mu, logvar = model(label_batch.float().cuda(), input_batch.float().cuda())
        val_loss = climate_loss_fn(output, label_batch.float().cuda())

    print(f'''
          Validation TVD Loss: {val_loss}''')

    torch.save(model.state_dict(), f"{folder}/checkpoint_{epoch+1}")
    print(f"Model weights saved to {folder} with point {epoch+1}.")
    
    # Inference

    initial_cond = torch.reshape(torch.tensor(psi[0,:]), [1, 75])
    time_step = 30000
    z = torch.zeros([1,latent_dim])
    num_ens = 1
    pred = np.zeros ([time_step, 75, num_ens])


    for k in range (0, time_step):

        for ens in range (0, num_ens):
            if (k ==0):

                z = torch.randn_like(z)
                print(z.shape, initial_cond.shape)
                y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()
                pred[k,:,ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

            else:
                select_ens = np.random.randint(0,num_ens,1)
                z = torch.randn_like(z)
                y = (model.decode(z.float().cuda(),torch.reshape(torch.tensor(pred[k-1,:,select_ens]),[1,75]).float().cuda())).detach().cpu().numpy()
                pred[k,:, ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

    # Denormalize final preds
    print(std_psi[:, 63])
    pred_mean = pred * std_psi[:, 63] + mean_psi[:, 63]

    # Denormalize test labels
    actual_values = psi_train_label[:time_step, 63] * std_psi[:, 63] + mean_psi[:, 63]
    print(actual_values)

    plt.plot(pred_mean[0:30000,63],'r')
    plt.plot(actual_values[0:30000],'b')
    plt.title(f"Predictions vs Actual | Batch size of {batch_size}")
    plt.savefig(f'{folder}/prediction_epoch_{epoch+1}.png')
    plt.show()
    plt.cla()

torch.save(model.state_dict(), f"{model_weights_path}")
print(f"Model weights saved to {folder} with point {epoch+1}.")

In [ ]:
# CCDF Error bars

import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import linregress
import seaborn as sns

CCDF_sum = 0

import datetime
import os

num_ccdfs = 10

for i in range(0, num_ccdfs):
    # Inference
    initial_cond = torch.reshape(torch.tensor(psi[0,:]), [1, 75])
    time_step = 300000
    z = torch.zeros([1,latent_dim])
    num_ens = 1
    pred = np.zeros ([time_step, 75, num_ens])

    for k in range (0, time_step):

        for ens in range (0, num_ens):
            if (k ==0):

                z = torch.randn_like(z)
                print(z.shape, initial_cond.shape)
                y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()
                pred[k,:,ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

            else:
                select_ens = np.random.randint(0,num_ens,1)
                z = torch.randn_like(z)
                y = (model.decode(z.float().cuda(),torch.reshape(torch.tensor(pred[k-1,:,select_ens]),[1,75]).float().cuda())).detach().cpu().numpy()
                pred[k,:, ens] = y
                y_denorm = (y * std_psi[:, :] + mean_psi[:, :])
                initial_cond = torch.tensor((y_denorm - mean_psi[:, :]) / std_psi[:, :])

    # Denormalize final preds
    print(std_psi[:, 63])
    pred_mean = pred * std_psi[:, 63] + mean_psi[:, 63]

    # Denormalize test labels
    actual_values = psi[:time_step, 63] * std_psi[:, 63] + mean_psi[:, 63]
    print(actual_values)

    # Load the data; shape = (300000, 2, 75)
    real_data = np.load(r"/home/constantino-daniel-boscu/Documents/research/AI-RES/modified-code-main3/data/actual/long_run_310k.npy")
    predictions = pred_mean.reshape(300000, 1, 75)

    if (CCDF):
        real_data_1d = real_data[:, 1, 63]  # Now shape is (309700,)
        predictions_1d = predictions[:, 0, 63]  # shape (300000,)

        # Define bounds (assuming they apply to both datasets)
        upper_bound = 53.8 / 2.8935
        lower_bound = 1.75 / 2.8935

        # Function to calculate transition durations
        def calculate_transition_durations(y_values, upper_bound, lower_bound):
            times_between_transitions = []
            transition_start = None
            above_upper = False
            below_lower = False

            for i in range(1, len(y_values)):
                if y_values[i] < lower_bound:  
                    below_lower = True
                    above_upper = False
                elif y_values[i] > upper_bound:  
                    if below_lower and transition_start is not None:
                        times_between_transitions.append(i - transition_start)
                        transition_start = None  
                    above_upper = True
                    below_lower = False

                if below_lower and transition_start is None:
                    transition_start = i

            return times_between_transitions

        # Compute transition durations for real data
        real_durations = calculate_transition_durations(real_data_1d, upper_bound, lower_bound)

        # Compute transition durations for predictions data
        pred_durations = calculate_transition_durations(predictions_1d, upper_bound, lower_bound)

        # Plot setup
        plt.figure(figsize=(10, 6))

        def bootstrap_ci(data, num_bootstraps=1000, confidence_level=0.95):
            sample_size = len(data)
            bootstrap_means = np.zeros(num_bootstraps)
            
            for i in range(num_bootstraps):
                bootstrap_sample = np.random.choice(data, size=sample_size, replace=True)
                bootstrap_means[i] = np.mean(bootstrap_sample)
            
            ci_lower = np.percentile(bootstrap_means, (1 - confidence_level) / 2 * 100)
            ci_upper = np.percentile(bootstrap_means, (1 + confidence_level) / 2 * 100)
            
            return np.mean(data), ci_lower, ci_upper

        # === REAL DATA CCDF AND FIT ===
        if len(real_durations) == 0:
            print("No transitions detected in real data with current bounds!")
        else:
            real_data_sorted = np.sort(real_durations)
            ccdf_real = 1 - np.arange(1, len(real_data_sorted) + 1) / len(real_data_sorted)

            valid_indices_real = ccdf_real > 0
            x_fit_real = real_data_sorted[valid_indices_real]
            y_fit_real = np.log(ccdf_real[valid_indices_real])

            slope_real, intercept_real, *_ = linregress(x_fit_real, y_fit_real)

            x_line_real = np.linspace(min(x_fit_real), max(x_fit_real), 40)
            y_line_real = np.exp(slope_real * x_line_real + intercept_real)

            # Create a grid of x values (time steps / durations)

            bootstrap_mean = []
            ci_lower_vals = []
            ci_upper_vals = []

            for x in x_line_real:
                valid_indices = (real_durations > x).astype(float)
                mean_val, lower, upper = bootstrap_ci(valid_indices)
                bootstrap_mean.append(mean_val)
                ci_lower_vals.append(lower)
                ci_upper_vals.append(upper)

            bootstrap_mean = np.array(bootstrap_mean)
            ci_lower_vals = np.array(ci_lower_vals)
            ci_upper_vals = np.array(ci_upper_vals)

            # Calculate error bars (difference from the bootstrap mean)
            error_lower = bootstrap_mean - ci_lower_vals
            error_upper = ci_upper_vals - bootstrap_mean

            plt.step(real_data_sorted, ccdf_real, where='post', label='Real Data CCDF', linewidth=2, linestyle='--')
            plt.errorbar(x_line_real, bootstrap_mean, yerr=[error_lower, error_upper],
                    fmt='o', color='blue', capsize=3, ecolor='blue', label='Real Bootstrap 95% CI')
            plt.plot(x_line_real, y_line_real, 'b-', label=f'Real Exp Fit (slope={slope_real:.4f})', linewidth=2)

        # === PREDICTIONS CCDF AND FIT ===
        if len(pred_durations) == 0:
            print("No transitions detected in predictions with current bounds!")
        else:
            pred_data_sorted = np.sort(pred_durations)
            ccdf_pred = 1 - np.arange(1, len(pred_data_sorted) + 1) / len(pred_data_sorted)

            valid_indices_pred = ccdf_pred > 0
            x_fit_pred = pred_data_sorted[valid_indices_pred]
            y_fit_pred = np.log(ccdf_pred[valid_indices_pred])

            slope_pred, intercept_pred, *_ = linregress(x_fit_pred, y_fit_pred)

            x_line_pred = np.linspace(min(x_fit_pred), max(x_fit_pred), 40)
            y_line_pred = np.exp(slope_pred * x_line_pred + intercept_pred)

            # Create a grid of x values (time steps / durations)

            bootstrap_mean = []
            ci_lower_vals = []
            ci_upper_vals = []

            for x in x_line_pred:
                valid_indices = (pred_durations > x).astype(float)
                mean_val, lower, upper = bootstrap_ci(valid_indices)
                bootstrap_mean.append(mean_val)
                ci_lower_vals.append(lower)
                ci_upper_vals.append(upper)

            bootstrap_mean = np.array(bootstrap_mean)
            ci_lower_vals = np.array(ci_lower_vals)
            ci_upper_vals = np.array(ci_upper_vals)

            # Calculate error bars (difference from the bootstrap mean)
            error_lower = bootstrap_mean - ci_lower_vals
            error_upper = ci_upper_vals - bootstrap_mean

            plt.step(pred_data_sorted, ccdf_pred, where='post', label='Predictions CCDF', linewidth=2, linestyle='-.', color='red')
            plt.errorbar(x_line_pred, bootstrap_mean, yerr=[error_lower, error_upper],
                    fmt='o', color='red', capsize=3, ecolor='red', label='Pred Bootstrap 95% CI')
            plt.plot(x_line_pred, y_line_pred, 'r-', label=f'Pred Exp Fit (slope={slope_pred:.4f})', linewidth=2)

            CCDF_sum += slope_pred

        # Plot labels and formatting
        plt.xlabel('Time Duration (Steps)')
        plt.ylabel('CCDF')
        plt.title('CCDF of Time Between B->A and A->B Transitions (Exponential Fit)')
        plt.yscale("log")  # y-axis log scale
        plt.xscale("linear")  # x-axis linear scale
        plt.grid()
        plt.legend()
        plt.tight_layout()
        save_path = os.path.join(folder, "CCDF")
        plt.savefig(save_path)
        plt.show()

        print(1/np.mean(real_data_sorted))

CCDF_avg = CCDF_sum/num_ccdfs
print(CCDF_avg)

In [ ]:
# Tests package

import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import linregress
import seaborn as sns
import random
if (plot_data):
    #note that the value 300000 will have to change depending on the real and predictions data length
    u_profile_real = real_data[:300000, 1, level]  # Match time length with predictions
    u_profile_pred = predictions[:, 0, level]
    time_steps = np.arange(len(u_profile_pred))

    # === Plot ===
    plt.figure(figsize=(14, 6))

    plt.plot(time_steps, u_profile_real, label='Real Data', alpha=0.7)
    plt.plot(time_steps, u_profile_pred, label='Predictions', linestyle='--')


    # Labels, legend, and formatting
    plt.xlabel('Time step')
    plt.ylabel('U (m/s)')
    plt.title(f'Time Series of U at Vertical Level {level}')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    save_path = os.path.join(folder, "timeseries")
    save_path = os.path.join(save_path, "real_prediction_plot")
    plt.savefig(save_path)
    plt.show()

if (CCDF):
    real_data_1d = real_data[:, 1, 61]  # Now shape is (309700,)
    predictions_1d = predictions[:, 0, 61]  # shape (300000,)

    # Define bounds (assuming they apply to both datasets)
    upper_bound = 53.8 / 2.8935
    lower_bound = 1.75 / 2.8935

    # Function to calculate transition durations
    def calculate_transition_durations(y_values, upper_bound, lower_bound):
        times_between_transitions = []
        transition_start = None
        above_upper = False
        below_lower = False

        for i in range(1, len(y_values)):
            if y_values[i] < lower_bound:  
                below_lower = True
                above_upper = False
            elif y_values[i] > upper_bound:  
                if below_lower and transition_start is not None:
                    times_between_transitions.append(i - transition_start)
                    transition_start = None  
                above_upper = True
                below_lower = False

            if below_lower and transition_start is None:
                transition_start = i

        return times_between_transitions

    # Compute transition durations for real data
    real_durations = calculate_transition_durations(real_data_1d, upper_bound, lower_bound)

    # Compute transition durations for predictions data
    pred_durations = calculate_transition_durations(predictions_1d, upper_bound, lower_bound)

    # Plot setup
    plt.figure(figsize=(10, 6))

    # === REAL DATA CCDF AND FIT ===
    if len(real_durations) == 0:
        print("No transitions detected in real data with current bounds!")
    else:
        real_data_sorted = np.sort(real_durations)
        ccdf_real = 1 - np.arange(1, len(real_data_sorted) + 1) / len(real_data_sorted)

        valid_indices_real = ccdf_real > 0
        x_fit_real = real_data_sorted[valid_indices_real]
        y_fit_real = np.log(ccdf_real[valid_indices_real])

        slope_real, intercept_real, *_ = linregress(x_fit_real, y_fit_real)

        x_line_real = np.linspace(min(x_fit_real), max(x_fit_real), 100)
        y_line_real = np.exp(slope_real * x_line_real + intercept_real)

        plt.step(real_data_sorted, ccdf_real, where='post', label='Real Data CCDF', linewidth=2, linestyle='--')
        plt.plot(x_line_real, y_line_real, 'b-', label=f'Real Exp Fit (slope={slope_real:.4f})', linewidth=2)

    # === PREDICTIONS CCDF AND FIT ===
    if len(pred_durations) == 0:
        print("No transitions detected in predictions with current bounds!")
    else:
        pred_data_sorted = np.sort(pred_durations)
        ccdf_pred = 1 - np.arange(1, len(pred_data_sorted) + 1) / len(pred_data_sorted)

        valid_indices_pred = ccdf_pred > 0
        x_fit_pred = pred_data_sorted[valid_indices_pred]
        y_fit_pred = np.log(ccdf_pred[valid_indices_pred])

        slope_pred, intercept_pred, *_ = linregress(x_fit_pred, y_fit_pred)

        x_line_pred = np.linspace(min(x_fit_pred), max(x_fit_pred), 100)
        y_line_pred = np.exp(slope_pred * x_line_pred + intercept_pred)

        plt.step(pred_data_sorted, ccdf_pred, where='post', label='Predictions CCDF', linewidth=2, linestyle='-.', color='red')
        plt.plot(x_line_pred, y_line_pred, 'r-', label=f'Pred Exp Fit (slope={slope_pred:.4f})', linewidth=2)

    # Plot labels and formatting
    plt.xlabel('Time Duration (Steps)')
    plt.ylabel('CCDF')
    plt.title('CCDF of Time Between B->A and A->B Transitions (Exponential Fit)')
    plt.yscale("log")  # y-axis log scale
    plt.xscale("linear")  # x-axis linear scale
    plt.grid()
    plt.legend()
    plt.tight_layout()
    save_path = os.path.join(folder, "CCDF")
    save_path = os.path.join(save_path, "CCDF_plot")
    plt.savefig(save_path)
    plt.show()
if (Bi_modal_distribution):
    zonal_wind_data_real = real_data[:, 1, 63]  # variable index 1 (e.g., zonal wind), level 60
    zonal_wind_data_predictions = predictions[:, 0, 63]  # variable index 0 (predictions), level 60

    print(f"Shape of zonal_wind_data_real: {zonal_wind_data_real.shape}")
    print(f"Shape of zonal_wind_data_predictions: {zonal_wind_data_predictions.shape}")

    # Plot the bimodal histogram
    plt.figure(figsize=(12, 6))

    # Create histograms (overlaid)
    sns.histplot(zonal_wind_data_real, bins=50, kde=True, color='black', alpha=0.6, element='step', label='Real Data')
    sns.histplot(zonal_wind_data_predictions, bins=50, kde=True, color='red', alpha=0.6, element='step', label='Predictions')

    # Customize plot labels and title
    plt.title('Distribution of Zonal Winds For Real Data and Predictions', fontsize=16)
    plt.xlabel('Zonal Wind (m/s)', fontsize=14)
    plt.ylabel('Frequency', fontsize=14)

    # Add vertical lines at means
    plt.axvline(np.mean(zonal_wind_data_real), color='black', linestyle='--', label=f'Real Mean: {np.mean(zonal_wind_data_real):.2f}')
    plt.axvline(np.mean(zonal_wind_data_predictions), color='red', linestyle='--', label=f'Pred Mean: {np.mean(zonal_wind_data_predictions):.2f}')

    # Final plot settings
    plt.legend(fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    save_path = os.path.join(folder, "bi_modal_distribution")
    save_path = os.path.join(save_path, "bi_modal_distribution_plot")
    plt.savefig(save_path)
    plt.show()
if (single_step_profiles):
    # Ensure save directory exists
    os.makedirs(SAVE_DIR, exist_ok=True)

    # === Load model weights ===
    model.load_state_dict(torch.load(MODEL_PATH))
    model.eval()

    # === Randomly sample time points from real data ===
    time_indices = random.sample(range(0, real_data.shape[0] - 2), NUM_SAMPLES)
    print(f"Randomly sampled time steps: {time_indices}")

    # === Time series visualization ===
    real_data_timeseries = real_data[:, 1, LEVEL]
    time_steps_all = np.arange(len(real_data_timeseries))

    plt.figure(figsize=(12, 6))
    plt.plot(time_steps_all, real_data_timeseries, label="Real Data at Level 61", color='blue')

    # Mark sample points
    for idx_num, idx in enumerate(time_indices):
        plt.axvline(x=idx, color='green', linestyle='--', linewidth=2)
    if len(time_indices) > 0:
        plt.axvline(x=time_indices[0], color='green', linestyle='--', linewidth=2, label='Sampled Points')

    plt.title("Real Data Time Series with Sampled Points Highlighted")
    plt.xlabel("Time Step")
    plt.ylabel("U (m/s) at Level 61")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()

    save_path = os.path.join(SAVE_DIR, "real_data_timeseries_with_samples.png")
    plt.savefig(save_path)
    plt.show()

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")
    # === Iterate over each sampled time point ===
    for i, time_step in enumerate(time_indices):
        next_time_step = time_step + 1

        # === Real data: current and next ===
        real_current = real_data[time_step, 1, :]       
        real_next = real_data[next_time_step, 1, :]      

        # === Normalize real_current and make prediction for next step ===
        initial_condition_normalized = (real_current.reshape(1, 75, 1) - mean_psi.reshape(1, -1, 1)) / std_psi.reshape(1, -1, 1)
        current_input = torch.tensor(initial_condition_normalized, dtype=torch.float32).to(device)

        with torch.no_grad():
            z = torch.randn_like(z)
            print(z.shape, initial_cond.shape)
            y = (model.decode(z.float().cuda(),initial_cond.float().cuda())).detach().cpu().numpy()

        # === Denormalize predicted next ===
        pred_next_denorm = y.squeeze() * std_psi.squeeze() + mean_psi.squeeze()

        # === Extract U, Re(Psi), Im(Psi) components ===
        # U profiles
        U_current_real = real_current[51:74]
        U_next_real = real_next[51:74]
        U_next_pred = pred_next_denorm[51:74]

        # Re(Psi) profiles
        RePsi_current_real = real_current[0:24]
        RePsi_next_real = real_next[0:24]
        RePsi_next_pred = pred_next_denorm[0:24]

        # Im(Psi) profiles
        ImPsi_current_real = real_current[25:50]
        ImPsi_next_real = real_next[25:50]
        ImPsi_next_pred = pred_next_denorm[25:50]

        # === Differences ===
        U_diff_real = U_next_real - U_current_real
        U_diff_pred = U_next_pred - U_current_real

        RePsi_diff_real = RePsi_next_real - RePsi_current_real
        RePsi_diff_pred = RePsi_next_pred - RePsi_current_real

        ImPsi_diff_real = ImPsi_next_real - ImPsi_current_real
        ImPsi_diff_pred = ImPsi_next_pred - ImPsi_current_real

        # === Create a single figure with 3 rows (U, Re(Psi), Im(Psi)) ===
        fig, axes = plt.subplots(3, 2, figsize=(16, 18))  # 3 rows, 2 columns (Profile and Difference)

        z_levels_U = np.linspace(0, 70, 23)
        z_levels_RePsi = np.linspace(0, 70, 24)
        z_levels_ImPsi = np.linspace(0, 70, 25)

        # --- U ---
        axes[0, 0].plot(U_current_real, z_levels_U, 'x-', label="Real Current")
        axes[0, 0].plot(U_next_real, z_levels_U, 'd-', label="Real Next")
        axes[0, 0].plot(U_next_pred, z_levels_U, 's--', label="Predicted Next")
        axes[0, 0].set_title(f"U Profiles @ Step {time_step}")
        axes[0, 0].set_xlabel("U (m/s)")
        axes[0, 0].set_ylabel("Vertical Levels (km)")
        axes[0, 0].legend()

        axes[0, 1].plot(U_diff_real, z_levels_U, 'xb', label="Real Δ (Next - Current)")
        axes[0, 1].plot(U_diff_pred, z_levels_U, 'o--r', label="Pred Δ (Next - Current)")
        axes[0, 1].set_title("U Difference (Next - Current)")
        axes[0, 1].set_xlabel("ΔU (m/s)")
        axes[0, 1].legend()

        # --- Re(Psi) ---
        axes[1, 0].plot(RePsi_current_real, z_levels_RePsi, 'x-', label="Real Current")
        axes[1, 0].plot(RePsi_next_real, z_levels_RePsi, 'd-', label="Real Next")
        axes[1, 0].plot(RePsi_next_pred, z_levels_RePsi, 's--', label="Predicted Next")
        axes[1, 0].set_title(f"Re(Psi) Profiles @ Step {time_step}")
        axes[1, 0].set_xlabel("Re(Psi)")
        axes[1, 0].set_ylabel("Vertical Levels (km)")
        axes[1, 0].legend()

        axes[1, 1].plot(RePsi_diff_real, z_levels_RePsi, 'xb', label="Real Δ (Next - Current)")
        axes[1, 1].plot(RePsi_diff_pred, z_levels_RePsi, 'o--r', label="Pred Δ (Next - Current)")
        axes[1, 1].set_title("Re(Psi) Difference (Next - Current)")
        axes[1, 1].set_xlabel("ΔRe(Psi)")
        axes[1, 1].legend()

        # --- Im(Psi) ---
        axes[2, 0].plot(ImPsi_current_real, z_levels_ImPsi, 'x-', label="Real Current")
        axes[2, 0].plot(ImPsi_next_real, z_levels_ImPsi, 'd-', label="Real Next")
        axes[2, 0].plot(ImPsi_next_pred, z_levels_ImPsi, 's--', label="Predicted Next")
        axes[2, 0].set_title(f"Im(Psi) Profiles @ Step {time_step}")
        axes[2, 0].set_xlabel("Im(Psi)")
        axes[2, 0].set_ylabel("Vertical Levels (km)")
        axes[2, 0].legend()

        axes[2, 1].plot(ImPsi_diff_real, z_levels_ImPsi, 'xb', label="Real Δ (Next - Current)")
        axes[2, 1].plot(ImPsi_diff_pred, z_levels_ImPsi, 'o--r', label="Pred Δ (Next - Current)")
        axes[2, 1].set_title("Im(Psi) Difference (Next - Current)")
        axes[2, 1].set_xlabel("ΔIm(Psi)")
        axes[2, 1].legend()

        # === Finalize and Save ===
        plt.suptitle(f"Single Step Profile Comparisons at Time Step {time_step}", fontsize=18)
        plt.tight_layout(rect=[0, 0.03, 1, 0.97])

        save_path = os.path.join(SAVE_DIR, f"Profile_Summary_point_{time_step}.png")
        plt.savefig(save_path)
        plt.show()

        print(f"Combined plot saved for sampled point {time_step}")

    # Final debug
    print("Finished processing all sampled points.")
        # Debugging prints
    print(predictions.shape) 
    print(real_data.shape)